# API Client Workflow Example

This notebook demonstrates a complete, secure workflow for interacting with the Capital.com API using the `get_client` context manager. All API operations are performed within a single `with` block to ensure that the session is created securely and closed automatically, even if errors occur.

### The workflow consists of the following steps:

1.  **Environment Setup:** We import the necessary libraries and configure the environment to access our project's source code.
2.  **Secure Client Initialization:** The `with get_client(...)` statement handles the entire lifecycle of the API client:
    - It loads credentials securely from the `.env` file.
    - It establishes a new, authenticated session with the API.
    - It provides a ready-to-use `client` object for the duration of the block.
    - It automatically and safely closes the session when the block is exited.
3.  **Session Verification:** Inside the block, we first call `client.ping()` to confirm that the connection is active and valid.
4.  **Market Discovery:** We then fetch the top-level market categories to get an overview of the available markets.
5.  **Detailed Instrument Data:** Finally, we retrieve all detailed information for a specific instrument (`US500`) to demonstrate a targeted data query.

In [ ]:
# --- Step 1: Environment Setup and Imports ---
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

# Add the project root to the Python path to allow module imports
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.capitalcom_bot.client_factory import get_client  # noqa: E402

# Configure logger and pandas for clean output
logger.remove()
logger.add(sys.stderr, level="INFO")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)


# --- Steps 2-5: Execute the entire API workflow within a single, secure context ---
try:
    # The 'with' statement ensures the client is created and the session is closed automatically.
    with get_client(demo_mode=True) as client:
        # --- Session Verification ---
        logger.info("--- Verifying session status ---")
        if client.ping():
            logger.success("Session is active and credentials are valid.")
        else:
            # If the ping fails, we stop the execution immediately.
            logger.error("Session is not active. Please check credentials.")
            raise ConnectionError("Failed to establish a valid API session.")

        # --- Market Discovery ---
        logger.info("\n--- Mapping market categories ---")
        categories_response = client.get_market_categories()
        df_categories = pd.DataFrame(
            [node.model_dump() for node in categories_response.nodes]
        )
        display(df_categories)

        # --- Detailed Instrument Data ---
        epic_to_query = "US500"
        logger.info(f"\n--- Fetching full details for {epic_to_query} ---")
        full_details = client.get_full_market_details(epic=epic_to_query)

        # Normalize the nested JSON into a flat structure and transpose for readability
        series_details = pd.json_normalize(full_details.model_dump()).T
        series_details.columns = ["Value"]
        display(series_details)

except Exception as e:
    logger.error(f"An error occurred during the API workflow: {e}", exc_info=True)

# After the 'with' block finishes, the session is guaranteed to be closed.